In [ ]:
import sys
import os
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import gc

In [2]:
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl

In [3]:
# Cargar el objeto AnnData
adata_16 = sc.read('/data/scratch/LAB/enric/zz_Spatial_Seq_Hudosn/Results/adata_16.h5ad')
adata_16.obs.head()

,in_tissue,array_row,array_col,thing,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,leiden
AAACACCAATAACTGC-1,1,59,19,a,4090,8.316545,18917.0,9.847869,34.883967,47.428239,57.287096,69.302744,2346.0,7.760893,12.401544,5120.0,8.541105,27.065601,5
AAACAGCTTTCAGAAG-1,1,43,9,a,3793,8.241176,17094.0,9.746541,32.871183,47.396747,58.020358,70.369720,866.0,6.765039,5.066105,5551.0,8.621914,32.473381,2
AAACAGGGTCTATATT-1,1,47,13,a,4309,8.368693,20349.0,9.920836,32.232542,46.523171,57.236228,69.266303,1625.0,7.393878,7.985651,6224.0,8.736329,30.586269,5
AAACCGGGTAGGTACC-1,1,42,28,a,4308,8.368461,19406.0,9.873389,30.289601,43.651448,54.988148,67.989282,1129.0,7.029973,5.817789,5133.0,8.543640,26.450583,4
AAACCGTTCGTCCAGG-1,1,52,42,a,1309,7.177782,3294.0,8.100162,42.714026,52.884032,62.112933,75.440194,247.0,5.513429,7.498482,607.0,6.410175,18.427444,3


In [4]:
adata_16

AnnData object with n_obs × n_vars = 1716 × 17354
    obs: 'in_tissue', 'array_row', 'array_col', 'thing', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'leiden'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'spatial', 'thing_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    layers: 'counts', 'normalized'
    obsp: 'connectivities', 'distances'

In [ ]:
import pandas as pd

# Ruta del archivo original
file_path = '/data/scratch/LAB/enric/zz_Spatial_Seq_Hudosn/Patient_16/Spatial_TCR/Patient16_output_counts_mixcr3.tsv'

# Cargar el archivo sin encabezado
clone_counts = pd.read_csv(file_path, sep="\t", header=None)

# Inspeccionar las primeras filas
print("Contenido original del archivo:")
print(clone_counts.head(10))  # Mostrar las primeras 10 filas para revisión

# Ajustar el encabezado
# Suponiendo que la primera fila debería ser los nombres de los clones y la segunda los barcodes
# Desplazar los datos hacia la izquierda
clone_counts.columns = ['temp'] + [f'clone{i}' for i in range(1, clone_counts.shape[1])]

# Mover el índice de barcode a la primera columna
clone_counts = clone_counts.drop(columns=['temp'])
clone_counts = clone_counts.reset_index(drop=True)  # Reiniciar el índice

# Guardar el DataFrame corregido
output_path = '/data/scratch/LAB/enric/zz_Spatial_Seq_Hudosn/Patient_16/Spatial_TCR/Patient16_output_counts_fixed.tsv'
clone_counts.to_csv(output_path, sep="\t", index=False)

# Imprimir el DataFrame ajustado
print("\nDataFrame ajustado y guardado en:")
print(output_path)
print(clone_counts.head(10))  # Mostrar las primeras 10 filas después del ajuste